In [ ]:
"""
Iris Classification - Neural Network vs. Simple Model

Autorzy:
- Kacper Sewruk s23466
- Michał Jastrzemski s26245


Ten notebook wczytuje zbiór danych Iris z pliku iris.csv, dokonuje podziału
na zbiory treningowy i testowy, normalizuje dane, a następnie trenuje dwie sieci neuronowe:
1. MLP (wielowarstwowy perceptron) do klasyfikacji gatunków Iris.
2. Model z pojedynczą warstwą Dense (odpowiednik regresji logistycznej), również w Keras.

Na koniec wyniki obu modeli są porównywane, a logi i wykresy zapisywane w katalogu `logs/`.
"""

import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Tworzymy katalog logs jeśli nie istnieje
os.makedirs('logs', exist_ok=True)

def load_iris_data(csv_path):
    """
    Wczytuje dane Iris z pliku CSV.

    Parametry
    ---------
    csv_path : str
        Ścieżka do pliku CSV z danymi Iris. Oczekiwane kolumny: 
        sepal_length, sepal_width, petal_length, petal_width, species

    Zwraca
    -------
    X : ndarray shape (150,4)
        Macierz cech (sepal_length, sepal_width, petal_length, petal_width).
    y : ndarray shape (150,)
        Wektor etykiet (Iris-setosa, Iris-versicolor, Iris-virginica).
    """
    data = pd.read_csv(csv_path)
    X = data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values
    y = data['species'].values
    return X, y

def preprocess_data(X, y):
    """
    Przetwarza dane:
    - Koduje etykiety gatunków kwiatów do form numerycznych (0,1,2).
    - One-hot encoding etykiet.
    - Dzieli dane na zbiory treningowy i testowy.
    - Normalizuje cechy do standaryzowanego rozkładu (średnia=0, odchylenie=1).

    Parametry
    ---------
    X : ndarray
        Macierz cech.
    y : ndarray
        Wektor etykiet tekstowych.

    Zwraca
    -------
    X_train, X_test, y_train, y_test : ndarray
        Dane treningowe i testowe oraz etykiety w formie one-hot.
    """
    le = LabelEncoder()
    y_num = le.fit_transform(y)
    y_cat = tf.keras.utils.to_categorical(y_num, num_classes=3)

    X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

def build_mlp_model(input_shape):
    """
    Buduje model MLP do klasyfikacji Iris:
    - 2 warstwy ukryte Dense (16 i 8 neuronów, ReLU)
    - Warstwa wyjściowa softmax (3 klasy)

    Parametry
    ---------
    input_shape : tuple
        Kształt danych wejściowych (np. (4,) dla Iris)

    Zwraca
    -------
    model : tf.keras.Model
        Skompilowany model MLP.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_logistic_like_model(input_shape):
    """
    Buduje prosty model z jedną warstwą Dense (3 neurony, softmax),
    działający analogicznie do regresji logistycznej dla 3 klas.

    Parametry
    ---------
    input_shape : tuple
        Kształt danych wejściowych (np. (4,) dla Iris)

    Zwraca
    -------
    model : tf.keras.Model
        Skompilowany model o jednej warstwie.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    """
    Trenuje i ocenia dwa modele:
    1. MLP model (wielowarstwowy perceptron)
    2. Model z jedną warstwą (logistic-like)

    Zapisywane są wyniki do plików:
    - logs/results_iris.txt
    - logs/training_history_iris.png (wykres z przebiegu uczenia MLP)
    
    Parametry
    ---------
    X_train, X_test, y_train, y_test : ndarray
        Dane treningowe i testowe oraz etykiety.
    """
    mlp_model = build_mlp_model((X_train.shape[1],))
    history = mlp_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=8, verbose=0)
    loss_mlp, acc_mlp = mlp_model.evaluate(X_test, y_test, verbose=0)

    log_model = build_logistic_like_model((X_train.shape[1],))
    log_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=8, verbose=0)
    loss_log, acc_log = log_model.evaluate(X_test, y_test, verbose=0)

    # Zapisywanie wyników
    with open('logs/results_iris.txt', 'w') as f:
        f.write(f"MLP Accuracy: {acc_mlp}\n")
        f.write(f"Logistic-like Accuracy: {acc_log}\n")

    # Wykres treningu MLP
    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='val accuracy')
    plt.legend()
    plt.title('Training History (Iris MLP)')
    plt.savefig('logs/training_history_iris.png')
    plt.show()

def main():
    """
    Funkcja główna:
    1. Wczytuje dane Iris z pliku CSV.
    2. Przetwarza dane (kodowanie, normalizacja, podział).
    3. Trenuje modele i porównuje wyniki.
    """
    X, y = load_iris_data('iris.csv')
    X_train, X_test, y_train, y_test = preprocess_data(X, y)
    train_and_evaluate_models(X_train, X_test, y_train, y_test)

main()
